# Paraphronima corneas

## Inference

First open your heatmap in napari. 

In [8]:
# import modules
import torchio as tio

# load in the nifti file with torchio
image_path = '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii'

image = tio.ScalarImage(image_path)



In [2]:
image

ScalarImage(shape: (1, 1289, 979, 1289); spacing: (0.00, 0.00, 0.00); orientation: LPS+; path: "/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii")

In [9]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [5]:
import pandas as pd
# 2nd scan
#RR = pd.read_csv('/home/jake/projects/dhr/FEG20190213_01_corneas_resample_ratio.csv')
# 3rd scan
# RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_sp_FEG221129_366_corneas_estimating_resample_ratio.csv')
RR = pd.read_csv('resample_ratio_measurements/resample_ratio_measurement_Paraphronima_crassipes_f536_u1701837_head.csv')

RR

,index,axis-0,axis-1,axis-2,axis-3
0,0.0,0.0,483.722771,475.212660,188.0
1,1.0,0.0,466.818301,506.629458,188.0
2,2.0,0.0,510.514762,494.987700,191.0
3,3.0,0.0,530.768231,465.484615,191.0
4,4.0,0.0,498.235099,522.258119,191.0
5,5.0,0.0,560.909220,463.730378,196.0
6,6.0,0.0,557.719698,480.634848,196.0
7,7.0,0.0,541.931560,517.952264,196.0
8,8.0,0.0,602.054063,473.298946,210.0
9,9.0,0.0,593.761304,511.732694,210.0


In [6]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix


In [7]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

,index,axis-0,axis-1,axis-2,axis-3,nearest_distance
0,0.0,0.0,483.722771,475.212660,188.0,33.434458
1,1.0,0.0,466.818301,506.629458,188.0,35.217471
2,2.0,0.0,510.514762,494.987700,191.0,29.907622
3,3.0,0.0,530.768231,465.484615,191.0,30.603212
4,4.0,0.0,498.235099,522.258119,191.0,29.907622
5,5.0,0.0,560.909220,463.730378,196.0,17.202737
6,6.0,0.0,557.719698,480.634848,196.0,17.202737
7,7.0,0.0,541.931560,517.952264,196.0,39.235015
8,8.0,0.0,602.054063,473.298946,210.0,39.318226
9,9.0,0.0,593.761304,511.732694,210.0,35.174883


In [8]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance

31.768202046528277

In [9]:
resample_ratio = mean_nearest_distance/10 #Makes average distance between features 10 voxels
resample_ratio

3.1768202046528278

## Inference

In [ ]:
!python main.py infer -h #If you need help for parameters in infer

usage: main.py [-h] [--volume_path VOLUME_PATH]
               [--sql_storage_url SQL_STORAGE_URL]
               [--starting_weights_path STARTING_WEIGHTS_PATH]
               [--model_path MODEL_PATH] [--profile] [--num_steps NUM_STEPS]
               [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
               [--resample_ratio_path RESAMPLE_RATIO_PATH]
               [--bbox_path BBOX_PATH] [--n_x_dirs N_X_DIRS]
               [--n_y_dirs N_Y_DIRS] [--n_z_dirs N_Z_DIRS]
               [--resample_ratio RESAMPLE_RATIO] [--already_resampled]
               [--training_data_histogram] [--infer_with_max_filter_fun]
               [--average_threshold AVERAGE_THRESHOLD]
               [--inference_peak_min_val INFERENCE_PEAK_MIN_VAL]
               {train,tune,infer,locate_peaks} config_path

Train, hyperparameter tune or infer with a deep heatmap regression model

positional arguments:
  {train,tune,infer,locate_peaks}
                        Mode of operation
  config_path      

Make inference first with 1 direction in the x, y, and z axis before creating inference in 3 directions. This will produce three files: two csv files (one for the resampled space, and the other not), and a nifty file with predictions.

*Note: these files are currently saved to ./dhr/output/

In [ ]:
#Inference with 1 direction in x, y, and z
# !python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 1 -ny 1 -nz 1

# Inference in 3 directions with 0.25 threshold
!python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 3 -ny 3 -nz 3 --average_threshold 0.25 -ipmv 0.25

TorchIO version: 0.18.88
Running setup!
Found 9 labelled images with labels in ./dataset/paraphronima_corneas/patches/train_images_10/ and ./dataset/paraphronima_corneas/patches/train_labels_10/ for analysis
Found 5 labelled images with labels in ./dataset/paraphronima_corneas/patches/test_images_10/ and ./dataset/paraphronima_corneas/patches/test_labels_10/ for analysis
Training:   8
Validation:  1
Test:       5
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'paraphronima_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.01757567945525256, 'heatmap_max_length': 64, 'histogram_landmarks_path': "
 "'./landmarks_files/paraphronima_corneas_landmarks.npy', "
 "'histogram_standardisation': False, 'ignore_empty_volumes': True, "
 "'image_suffix': 'image', 'label_suffix': 'corneas', 'learn_sigma': False, "


You can then view your inference in napari. It might be useful to you, especially if you are trying many inferences with modified parameters, to evalute the performance of each and track estimated % of correct and incorrect inference points in an Excel spreadsheet for example.

In [11]:
import napari
import numpy as np
import torchio as tio


image = tio.ScalarImage(image_path)

heatmap_path = './output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction.nii'
#Make sure to use resampled space csv here.
y_hat_path = 'output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction_peak_min_val_0_25_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
#viewer.add_image(image.numpy(), name='image')

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)


<Points layer 'Points' at 0x7fe2301a3fa0>